##  Overview

- Create config file for prometheus
- Create config file for influx db
- Create volume with config files
- Deploy Influx
- Deploy Prometheus
- Create some basic alert when a target is missing
- Test

### Get repo with docker-compose files

Source [6za/monitoring-tools](https://github.com/6za/monitoring-tools)

In [170]:
%%bash

mkdir ~/repos
cd ~/repos
#rm -rf ~/repos/monitoring-tools
git clone https://github.com/6za/monitoring-tools.git
ls  ~/repos/monitoring-tools/prometheus

prometheus-template.yml


mkdir: cannot create directory '/root/repos': File exists
Cloning into 'monitoring-tools'...


### Deploy Influx

- Select node o host
- Deploy it 
- Create prometheusdb

Reference: 
- https://hub.docker.com/_/influxdb
- https://docs.influxdata.com/influxdb/v1.7/supported_protocols/prometheus/

In [171]:
import pandas as pd
hosts = pd.read_csv("../common/hosts.csv")
supressed_columns = ['ip','user']

In [172]:
influx_hosts = hosts[(hosts.hostname == "pi-node7" )]
influx_hosts['ip'].to_csv(r'influx-ip.txt', header=False, index=None, sep=' ')

In [173]:
%%bash
export HOME_DIR=$PWD
echo $HOME_DIR
source ../common/env.sh 
while read ip; do
    cd ~/repos/monitoring-tools
    export DOCKER_HOST="tcp://$ip:2376"
    docker-compose -f docker-compose-influx.yaml down 2>/dev/null
    docker-compose -f docker-compose-influx.yaml up -d 2>/dev/null
    # Wait DB is ready
    while [[ "$(curl -s -o /dev/null -w '%{http_code}' -XPOST http://$ip:8086/query --data-urlencode 'q=CREATE DATABASE prometheus' )" != "200" ]]; do
        echo "Waiting influx to be ready..."
        sleep 15
    done
done <influx-ip.txt
cd $HOME_DIR

/root/05-monitoring-stack
Waiting influx to be ready...
Waiting influx to be ready...


Create DB on influxDB

InfluxDB created and DB created for `prometheus` metrics. This call need to return 200, to confirm creation was done.


### Deploy Prometheus

- Create Prometheus targets files based on a template
- Fill template with hosts/monitors known
- Create volume with configs
- Deploy docker-compose with proper volume mounts

In [174]:
import yaml


with open('/root/repos/monitoring-tools/prometheus/prometheus-template.yml',"r") as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    config = yaml.load(file, Loader=yaml.FullLoader)

    influx_config = influx_hosts.head()['ip'].iloc[0]
    config['remote_write'] = [{'url': "http://"+influx_config+ ":8086/api/v1/prom/write?db=prometheus"}]
    config['remote_read']= [{'url': "http://"+influx_config+ ":8086/api/v1/prom/read?db=prometheus"}]
    #Simple Node Exporter
    for index, row in hosts.iterrows():
        docker_host_config = {'job_name': 'node-exporter-'+row['hostname'], 'metrics_path': '/metrics', 'static_configs': [{'targets': [row['ip']+':9100']}]}
        config['scrape_configs'].append(docker_host_config)
    #Simple Docker-Host Exporter       
    for index, row in hosts.iterrows():
        docker_host_config = {'job_name': 'docker-hosts-exporter-'+row['hostname'], 'static_configs': [{'targets': [row['ip']+':9323']}]}
        config['scrape_configs'].append(docker_host_config)    
    #sensor-exporter  on i86 - port 9255
    for index, row in hosts[(hosts.arch == "x86_64")].iterrows():                
        docker_host_config = {'job_name': 'sensor-exporter-'+row['hostname'], 'metrics_path': '/metrics', 'static_configs': [{'targets': [row['ip']+':9255']}]}
        config['scrape_configs'].append(docker_host_config)    
    with open(r'prometheus.yml', 'w') as outputfile:
        documents = yaml.dump(config, outputfile)

#### Create Volume on all nodes running prometheus
> There is a weird version on dockerhub that breaks if used the last tag. 
> the known working tag for arm is `prom/prometheus:v2.12.0`

In [175]:
#hosts[(hosts.arch == "x86_64") & (hosts.hostname == "nuc-01" )]['ip'].to_csv(r'prom-ip.txt', header=False, index=None, sep=' ')
hosts[(hosts.hostname == "pi-node7" )]['ip'].to_csv(r'prom-ip.txt', header=False, index=None, sep=' ')

In [176]:
%%bash
export HOME_DIR=$PWD
echo $HOME_DIR
source ../common/env.sh 
while read ip; do
    cd ~/repos/monitoring-tools
    export DOCKER_HOST="tcp://$ip:2376"
    docker-compose -f docker-compose-prometheus.yaml down 2>/dev/null
done <prom-ip.txt
cd $HOME_DIR

/root/05-monitoring-stack


In [177]:
%%bash
source ../common/env.sh 
while read ip; do
    export DOCKER_HOST="tcp://$ip:2376"
    #Create Volume 
    docker volume rm prometheus-conf
    docker volume create --name prometheus-conf
    docker run -d -v prometheus-conf:/root/conf --name init-conf-prom busybox tail -f /dev/null
    # populate volume
    docker cp prometheus.yml init-conf-prom:/root/conf
    docker stop init-conf-prom
    docker rm init-conf-prom
done <prom-ip.txt

prometheus-conf
prometheus-conf
676defce69e4ecb33d1e5df0aa2f09a20ed876bc208192168d6dec1942414b3b
init-conf-prom
init-conf-prom


#### Deploy Prometheus on all target nodes

In [178]:
%%bash
export HOME_DIR=$PWD
echo $HOME_DIR
source ../common/env.sh 
while read ip; do
    cd ~/repos/monitoring-tools
    export DOCKER_HOST="tcp://$ip:2376"
    docker-compose -f docker-compose-prometheus.yaml up -d 2>/dev/null
done <prom-ip.txt
cd $HOME_DIR

/root/05-monitoring-stack


In [179]:
hosts['ip'].to_csv(r'ip.txt', header=False, index=None, sep=' ')

In [180]:
%%bash
source ../common/env.sh 
echo "ip|name|id|image|ports|runnning" > containers.csv
while read ip; do
    export DOCKER_HOST="tcp://$ip:2376"
    docker ps  --format "$ip|{{.Names}}|{{.ID}}|{{.Image}}|{{.Ports}}|{{.RunningFor}}"  >> containers.csv 2>/dev/null || :
done <ip.txt

In [181]:
containers = pd.read_csv("containers.csv", sep = '|')
pd.merge(containers, hosts, on='ip', how='inner').drop(columns=supressed_columns)

,name,id,image,ports,runnning,hostname,arch
0,nodeexporter,fb1114ef5ff5,prom/node-exporter:v0.18.0,0.0.0.0:9100->9100/tcp,18 hours ago,pi-node2,armv7l
1,nodeexporter,fbdf7410748b,prom/node-exporter:v0.18.0,0.0.0.0:9100->9100/tcp,18 hours ago,pi-node3,armv7l
2,nodeexporter,c5c1b27b0828,prom/node-exporter:v0.18.0,0.0.0.0:9100->9100/tcp,18 hours ago,kx-Lenovo-H520g,x86_64
3,node-exporter-collectors_tempsensor_1,4f9ee299d149,amkay/sensor-exporter,0.0.0.0:9255->9255/tcp,18 hours ago,kx-Lenovo-H520g,x86_64
4,prometheus,3e719b0fad03,prom/prometheus,NaN,7 months ago,kx-Lenovo-H520g,x86_64
5,grafana,9ea5ac9a3688,grafana/grafana,NaN,7 months ago,kx-Lenovo-H520g,x86_64
6,registry,1b947aee9a12,registry:2,0.0.0.0:5000->5000/tcp,12 months ago,kx-Lenovo-H520g,x86_64
7,prometheus,f86073a60478,prom/prometheus:v2.12.0,0.0.0.0:9090->9090/tcp,47 minutes ago,nuc-01,x86_64
8,node-exporter-collectors_tempsensor_1,0e0406780847,amkay/sensor-exporter,0.0.0.0:9255->9255/tcp,18 hours ago,nuc-01,x86_64
9,nodeexporter,bfb8d0b8a9fa,prom/node-exporter:v0.18.0,0.0.0.0:9100->9100/tcp,18 hours ago,nuc-01,x86_64


If all worked as expected you should see a stream of logs at influxdb consuming data from prometheus.

Use: `docker logs influxdb -f` 